In [ ]:
from IPython.display import display
import pandas as pd


def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

# Préambule : Marseille se trouve à deux endroits dans les données toflit18 car c'est un port franc !

On trouve Marseille comme direction des fermes rapportrices d'échanges entre la France et un partenaire étranger (`customs_region` = 'Marseille) et comme partenaire "à l'instar de l'étranger" d'échanges avec des régions françaises  (`partner_simplification` = 'Marseille').

# Quelques premières analyses sur Marseille comme direction des fermes

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()


marseille_as_direction_flows = toflit_client.get_flows(
 customs_region = 'Marseille',
)
print(str(len(marseille_as_direction_flows)) + " flux (observations)")

In [ ]:
marseille_as_direction_flows[0]

In [ ]:
sources_as_direction = {}
for p in marseille_as_direction_flows:
    source = p["source_type"]
    action = p["export_import"]
    value = float('.'.join(p["value"].split('.')[0:1]) or 0)
    year = p["year"]
    if source not in sources_as_direction:
        sources_as_direction[source] = {}
    if year not in sources_as_direction[source]:
        sources_as_direction[source][year] = {}
    if action not in sources_as_direction[source][year]:
        sources_as_direction[source][year][action] = {
            "nb_flows": 0,
            "value": 0
        }
    sources_as_direction[source][year][action]["value"] += value
    sources_as_direction[source][year][action]["nb_flows"] += 1

sources_as_direction_list = []
for source, yeargroup in sources_as_direction.items():
    for year, actionsgroup in yeargroup.items():
        for action, dati in actionsgroup.items():
            sources_as_direction_list.append({
                "source": source,
                "year": year,
                "action": action,
                "nb_flows": dati["nb_flows"],
                "value": dati["value"],
            })

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de flux par type, année et source pour Marseille comme direction des fermes",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "rect"
    },
    "width": 200,
    "data": {
        "values": sources_as_direction_list
    },
    "encoding": {
        "column": {
            "field": "source"
        },
        "x": {
            "field": "action",
            "type": "nominal", 
            "title": "action",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "year",
            "type": "nominal",
            "title": "année"
        },
        "color": {
            "field": "nb_flows",
            "type": "quantitative",
            "title": "Nombre de flux"
        }
    }
})

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de flux par type, année et source pour Marseille comme direction des fermes",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "rect"
    },
    "width": 200,
    "data": {
        "values": sources_as_direction_list
    },
    "encoding": {
        "column": {
            "field": "source"
        },
        "x": {
            "field": "action",
            "type": "nominal", 
            "title": "action",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "year",
            "type": "nominal",
            "title": "année"
        },
        "color": {
            "field": "value",
            "type": "quantitative",
            "title": "Valeur en lt."
        }
    }
})

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Détails du nombre d'observations des bureaux de ferme par source",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "rect"
    },
    "width": 200,
    "data": {
        "values": [{**f, "value": float('.'.join(f["value"].split('.')[0:1]) or 0)} for f in marseille_as_direction_flows]
    },
    "encoding": {
        "column": {
            "field": "source_type"
        },
        "x": {
            "field": "export_import",
            "type": "nominal", 
            "title": "action",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "customs_office_source",
            "type": "nominal",
            "title": "bureau des fermes"
        },
        "color": {
            "aggregate": "count",
            "type": "quantitative",
            "title": "Nombre de flux"
        }
    }
})

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Couverture diachronique des bureaux de ferme par source (et nombre d'observations)",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "rect"
    },
    "width": 200,
    "data": {
        "values": [f for f in marseille_as_direction_flows if f['customs_office_source'] != '']
    },
    "encoding": {

        "column": {
            "field": "customs_office_source"
        },
        "x": {
            "field": "export_import",
            "type": "nominal", 
            "title": "action",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "year",
            "type": "nominal",
            "title": "année"
        },
        "color": {
            "aggregate": "count",
            "type": "quantitative",
            "title": "Nombre de flux"
        }
    }
})

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Détails du nombre d'observations des bureaux de ferme par source",
        
    "width": 200,
    "data": {
        "values": marseille_as_direction_flows
    },
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "rect"
    },
    "encoding": {
        "column": {
            "field": "export_import"
        },
        "x": {
            "field": "product_RE_aggregate",
            "type": "nominal", 
            "title": "produit (RE aggregate)",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "partner_grouping",
            "type": "nominal",
            "title": "partenaire (grouping)"
        },
        "color": {
            "aggregate": "sum",
            "field": "value",
            "type": "quantitative",
            "title": "valeur cumulée en lt"
        }
    }
        
})

# Quelques premières analyses sur Marseille comme partenaire

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()


marseille_as_partner_flows = toflit_client.get_flows(
 partner_simplification = 'Marseille',
)
print(str(len(marseille_as_partner_flows)) + " observations (flows)")

In [ ]:
sources_as_partner = {}
for p in marseille_as_partner_flows:
    source = p["source_type"]
    action = p["export_import"]
    year = p["year"]
    if source not in sources_as_partner:
        sources_as_partner[source] = {}
    if year not in sources_as_partner[source]:
        sources_as_partner[source][year] = {}
    if action not in sources_as_partner[source][year]:
        sources_as_partner[source][year][action] = 0
    sources_as_partner[source][year][action] += 1

sources_as_partner_list = []
for source, yeargroup in sources_as_partner.items():
    for year, actionsgroup in yeargroup.items():
        for action, count in actionsgroup.items():
            sources_as_partner_list.append({
                "source": source,
                "year": year,
                "action": action,
                "count": count
            })

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de flux par type, année et source pour Marseille comme partenaire",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "rect"
    },
    "width": 300,
    "data": {
        "values": sources_as_partner_list
    },
    "encoding": {
        "column": {
            "field": "source"
        },
        "x": {
            "field": "action",
            "type": "nominal", 
            "title": "action",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "year",
            "type": "nominal",
            "title": "année"
        },
        "color": {
            "field": "count",
            "type": "quantitative",
            "title": "Nombre de flux"
        }
    }
})